In [85]:
import json
from pprint import pprint

# load bmeg output
individual_treatments = json.load(open('/util/bmeg/biostream/biostream/g2p/individual-treatments.json'))

In [86]:
# load associations index by gene and compound
gene_associations = {}
compound_associations = {}
with open('/util/bmeg/biostream/biostream/g2p/G2PAssociation.json') as f:
    for line in f:
        association = json.loads(line)
        # create a lightweight association
        _association = {'id': association['id'],
                        'description': association.get('description', None),
                        'genes': association.get('genes', []),
                        'environments': ['compound:{}'.format(compound) for compound in association.get('environments', [])]  }
        for gene in association.get('genes', []):
          if gene not in gene_associations:
            gene_associations[gene] = []
          gene_associations[gene].append(_association)
        for compound in association.get('environments', []):
          k = 'compound:{}'.format(compound)
          if k not in compound_associations:
            compound_associations[k] = []
          compound_associations[k].append(_association)

In [87]:
# for unsupported compounds, ensure all gene associations do not have compound
unknown_individual_compounds = set([])
for c,individual_treatment in enumerate(individual_treatments):
    unsupporteds = individual_treatment.get('unsupported',None) or []
    for unsupported in unsupporteds:
        if unsupported not in compound_associations:
            individual_treatment_variants = set(individual_treatment.get('variants',None) or [])
            _compounds = []
            for individual_treatment_variant in individual_treatment_variants:
                if individual_treatment_variant in gene_associations:
                    _compounds.extend(gene_associations[individual_treatment_variant]['environments'])
            if unsupported not in _compounds:        
                unknown_individual_compounds.add(unsupported)
unknown_individual_compounds

{u'compound:CID38904',
 u'compound:CID5353562',
 u'compound:UNKNOWN:ciclosporin',
 u'compound:UNKNOWN:oxaliplatin',
 u'compound:UNKNOWN:temsirolimus'}

In [88]:
# for unsupported compounds, run it again to see if others
for c,individual_treatment in enumerate(individual_treatments):
    unsupporteds = individual_treatment.get('unsupported',None) or []
    for unsupported in unsupporteds:
        if unsupported in unknown_individual_compounds:
            continue
        if unsupported not in compound_associations:
            print '?', unsupported
